In [4]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score
dicio = catalog.load('pred_geral_lista')
 

[02/02/24 08:16:09] INFO     Loading data from 'pred_geral_lista' (PartitionedDataset)...       ]8;id=489516;file://C:\Users\ramon.boucas\.conda\envs\my_env\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=558409;file://C:\Users\ramon.boucas\.conda\envs\my_env\lib\site-packages\kedro\io\data_catalog.py#502\502]8;;\

In [5]:
lista = []

for key, value in dicio.items():
    df = value()
    df['modelo'] = key
    lista.append(df)


dfzao = pd.concat(lista)
dfzao

,index,sindex,tema,conjunto,SOMA_PREDS,PRED_GERAL,TARGET,modelo
0,0,14,2,1,735,731,840.0,TF_IDF_32
1,1,104,39,1,772,725,840.0,TF_IDF_32
2,2,80,13,1,703,703,520.0,TF_IDF_32
3,3,77,2,1,615,604,920.0,TF_IDF_32
4,4,40,54,1,763,709,640.0,TF_IDF_32
...,...,...,...,...,...,...,...,...
1521,1521,77,46,1,724,762,920.0,TF_IDF_64
1522,1522,16,43,1,707,749,680.0,TF_IDF_64
1523,1523,54,45,1,642,754,760.0,TF_IDF_64
1524,1524,125,81,1,677,717,760.0,TF_IDF_64


In [11]:
elef = dfzao.copy()

In [12]:
cohen_kappa_score(elef['SOMA_PREDS'],elef['TARGET'])

-0.00017315860876832012

In [13]:
cohen_kappa_score(elef['PRED_GERAL'],elef['TARGET'])

0.0009445858059033974

In [14]:
a = dicio['TF_IDF_32']()
b = dicio['TF_IDF_64']()

In [15]:
cohen_kappa_score(a['PRED_GERAL'],a['TARGET'])

-6.674081355750872e-05

In [16]:
cohen_kappa_score(b['PRED_GERAL'],b['TARGET'])

0.0019551957261074726